In [1]:
import pandas as pd
import os, sys
import pickle
import torch
import numpy as np
from dataloader.utils import task_splitter_circle_arrow, task_splitter

2025-08-17 10:43:47.592323: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-17 10:43:48.784827: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2025-08-17 10:43:48.785021: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local

In [33]:
clients, aug, batch_size = (7,True,16)

In [34]:
train_task_cl, incremental_test_loaders, all_test_loader, y_labels, raw_df = task_splitter('data', clients, aug, batch_size)

In [27]:
train_task_cl, incremental_test_loaders, all_test_loader, y_labels, raw_df = task_splitter_circle_arrow('Data', clients, aug, batch_size)
raw_df = raw_df.drop('Using circle', axis='columns')
raw_df = raw_df.rename(columns={'Using arrow':'domain'})

In [31]:

print("Total rows in raw_df:", len(raw_df))

# 1. Task splits by 'domain'
print("\nRaw dataframe counts per domain:")
print(raw_df['domain'].value_counts())

# 2. Count total number of samples actually present in each task/client split
for task_idx, task_loaders in enumerate(train_task_cl):
    total_split_samples = 0
    print(f"\nTask {task_idx}:")
    for client_idx, loader in enumerate(task_loaders):
        ds_len = len(loader.dataset)
        total_split_samples += ds_len
        print(f"  Client {client_idx}: Loader dataset size = {ds_len}")

    # Compare to corresponding raw domain count (after augmentation, if any)
    domain_name = raw_df['domain'].unique()[task_idx]
    true_raw_count = raw_df[raw_df['domain'] == domain_name].shape[0]
    print(f"  Total samples in split loaders = {total_split_samples}")
    print(f"  Raw dataframe count for domain '{domain_name}' = {true_raw_count}")

    # If augmenting, you should expect total_split_samples == true_raw_count * N_AUG
    # Otherwise, expect equality.

    if total_split_samples == true_raw_count:
        print("  ✓ Split count matches raw count.")
    elif total_split_samples % true_raw_count == 0:
        aug_factor = total_split_samples // true_raw_count
        print(f"  ✓ Split is augmented. Augmentation factor: {aug_factor}")
    else:
        print("  ⚠ Warning: Split count ({}) does not match or scale cleanly from raw count ({}) !"
              .format(total_split_samples, true_raw_count))

# 3. Global check: sum all split samples vs. entire raw dataframe
split_total = sum(len(loader.dataset) for task_loaders in train_task_cl for loader in task_loaders)
print("\nTotal samples across all tasks/clients:", split_total)
print("Total rows in raw_df:", len(raw_df))
if split_total == len(raw_df):
    print("Overall split matches raw data rows")
elif split_total > len(raw_df):
    print("Overall split is larger than raw data (augmentation/duplication)")
else:
    print("Overall split is smaller than raw data! Potential data loss.")

# 4. You can also print the DataLoader batch counts in each split
for task_idx, task_loaders in enumerate(train_task_cl):
    print('')
    for client_idx, loader in enumerate(task_loaders):
        print(f"Task {task_idx}, Client {client_idx}: Num batches = {len(loader)}, Batch size = {loader.batch_size}")

Total rows in raw_df: 1600

Raw dataframe counts per domain:
Home           800
BigOffice-2    160
BigOffice-3    160
Hallway        160
MeetingRoom    160
SmallOffice    160
Name: domain, dtype: int64

Task 0:
  Client 0: Loader dataset size = 800
  Client 1: Loader dataset size = 800
  Client 2: Loader dataset size = 800
  Client 3: Loader dataset size = 800
  Client 4: Loader dataset size = 800
  Client 5: Loader dataset size = 800
  Client 6: Loader dataset size = 800
  Client 7: Loader dataset size = 800
  Client 8: Loader dataset size = 800
  Client 9: Loader dataset size = 800
  Total samples in split loaders = 8000
  Raw dataframe count for domain 'Home' = 800
  ✓ Split is augmented. Augmentation factor: 10

Task 1:
  Client 0: Loader dataset size = 160
  Client 1: Loader dataset size = 160
  Client 2: Loader dataset size = 160
  Client 3: Loader dataset size = 160
  Client 4: Loader dataset size = 160
  Client 5: Loader dataset size = 160
  Client 6: Loader dataset size = 160


In [35]:
print("Number of train tasks:", len(train_task_cl))
for task_idx, task_loaders in enumerate(train_task_cl):
    print(f"Task {task_idx}:")
    print("  Number of clients:", len(task_loaders))
    for client_idx, loader in enumerate(task_loaders):
        print(f"    Client {client_idx}: Dataset size = {len(loader.dataset)}, Number of batches = {len(loader)}")
    total_samples = sum([len(loader.dataset) for loader in task_loaders])
    print(f"  Total samples for task {task_idx}: {total_samples}")

print("\nIncremental Test Loaders:")
for i, loader in enumerate(incremental_test_loaders):
    print(f"  Test loader {i}: Dataset size = {len(loader.dataset)}, Batches = {len(loader)}")

print("\nAll Test Loader:")
print(f"  Dataset size = {len(all_test_loader.dataset)}, Batches = {len(all_test_loader)}")

print("\nLabel columns:", y_labels)


Number of train tasks: 6
Task 0:
  Number of clients: 7
    Client 0: Dataset size = 1142, Number of batches = 72
    Client 1: Dataset size = 1142, Number of batches = 72
    Client 2: Dataset size = 1142, Number of batches = 72
    Client 3: Dataset size = 1142, Number of batches = 72
    Client 4: Dataset size = 1142, Number of batches = 72
    Client 5: Dataset size = 1142, Number of batches = 72
    Client 6: Dataset size = 1148, Number of batches = 72
  Total samples for task 0: 8000
Task 1:
  Number of clients: 7
    Client 0: Dataset size = 228, Number of batches = 15
    Client 1: Dataset size = 228, Number of batches = 15
    Client 2: Dataset size = 228, Number of batches = 15
    Client 3: Dataset size = 228, Number of batches = 15
    Client 4: Dataset size = 228, Number of batches = 15
    Client 5: Dataset size = 228, Number of batches = 15
    Client 6: Dataset size = 232, Number of batches = 15
  Total samples for task 1: 1600
Task 2:
  Number of clients: 7
    Client 

In [ ]:
Number of train tasks: 2
Task 0:
  Number of clients: 10
    Client 0: Dataset size = 294, Number of batches = 18
    Client 1: Dataset size = 294, Number of batches = 18
    Client 2: Dataset size = 294, Number of batches = 18
    Client 3: Dataset size = 294, Number of batches = 18
    Client 4: Dataset size = 294, Number of batches = 18
    Client 5: Dataset size = 294, Number of batches = 18
    Client 6: Dataset size = 294, Number of batches = 18
    Client 7: Dataset size = 294, Number of batches = 18
    Client 8: Dataset size = 294, Number of batches = 18
    Client 9: Dataset size = 294, Number of batches = 18
  Total samples for task 0: 2940
Task 1:
  Number of clients: 10
    Client 0: Dataset size = 267, Number of batches = 16
    Client 1: Dataset size = 267, Number of batches = 16
    Client 2: Dataset size = 267, Number of batches = 16
    Client 3: Dataset size = 267, Number of batches = 16
    Client 4: Dataset size = 267, Number of batches = 16
    Client 5: Dataset size = 267, Number of batches = 16
    Client 6: Dataset size = 267, Number of batches = 16
    Client 7: Dataset size = 267, Number of batches = 16
    Client 8: Dataset size = 267, Number of batches = 16
    Client 9: Dataset size = 267, Number of batches = 16
  Total samples for task 1: 2670

Incremental Test Loaders:
  Test loader 0: Dataset size = 99, Batches = 6
  Test loader 1: Dataset size = 90, Batches = 5

All Test Loader:
  Dataset size = 189, Batches = 11